<a href="https://colab.research.google.com/github/raitharnett/tensorflow-great-barrier-reef/blob/main/cots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
%alias install_kaggle mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%alias download_cots_data kaggle competitions download -c tensorflow-great-barrier-reef --force >& /dev/null

In [ ]:
%alias install_cots_data unzip -d  tensorflow-great-barrier-reef tensorflow-great-barrier-reef.zip >& /dev/null

In [ ]:
# download tensorflow-great-barrier-reef data and import library used to submit predictions
from google.colab import files
import os
import sys
COTS_DATA = '/content/tensorflow-great-barrier-reef'
if not os.path.isdir(COTS_DATA):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  %install_kaggle
  %download_cots_data
  %install_cots_data
sys.path.insert(0, COTS_DATA)
import greatbarrierreef

In [ ]:
import contextlib2
import io
import IPython
import json
import numpy as np
import os
import pathlib
import pandas as pd
import sys
import tensorflow as tf
import time

from PIL import Image, ImageDraw

In [ ]:
data_df = pd.read_csv(os.path.join(COTS_DATA,'train.csv'))
